# First Steps with Prompt Engineering

In [2]:
import os
from openai import OpenAI
import cohere

In [ ]:
co = cohere.Client("")

openai_client = OpenAI(
    api_key= ""
)

In [4]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given model "
        
    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                #role label who is speaking(user:human, assistance: model, tool:external tool)
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [5]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.generations[0].text}')

## Just ASK

In [7]:
test_prompt_openai('Translate to traditional Chinese.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to traditional Chinese.

Where is the nearest restaurant?
------
RESPONSE
------
最近的餐廳在哪裡？


In [9]:
test_prompt_cohere('Translate the following setence to traditional Chinese.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate the following setence to traditional Chinese.

Where is the nearest restaurant?
------
RESPONSE
------
 我該找最近的餐廳嗎?

In traditional Chinese, this sentence translates to: 我要找最近的餐厅是什么地方? 

Let me know if you would like me to translate this for you in any other languages. 


In [17]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
 En iyi restoranın nearest locationından hangi yeşillerdir?

Bu bir soruyu Türkçe çeviriye başlayan kişinin sorunu yaşamaya çalışabilir. Bir restaurantın en yakın locationında hangi yeşillerde bulunabilirler, gibi bir sorunu gerçek bir çevirisi olmadan karşılaşabilir. 

If you are trying to locate the nearest restaurant in Turkish, the phrase you are looking for is: 

"En iyi restoranın nearest locationından hangi yeşillerdir?" 

This translates to "Which directions are the nearest restaurant's location from here?" 

I hope this helps, let me know if you would like to translate anything else! 


# Few-shot learning

Using examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](../images/gpt3_paper.png)

In [11]:
examples = [
    ('Review: This movie sucks\nSubjective: oh! Yes'),
    ('Review: This tv show was about the ocean\nSubjective: oh! No'),
    ('Review: This book had a lot of flaws\nSubjective:oh! Yes'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: oh! Yes
###
Review: This tv show was about the ocean
Subjective: oh! No
###
Review: This book had a lot of flaws
Subjective:oh! Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
oh! No


In [12]:
# Cohere is getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: oh! Yes
###
Review: This tv show was about the ocean
Subjective: oh! No
###
Review: This book had a lot of flaws
Subjective:oh! Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 oh! No


In [13]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------
I found the book to be a compelling and informative look at the events of WWII. The author did a great job of presenting the historical context and key figures involved in the war. Overall, I would highly recommend this book to anyone interested in learning more about this important period in history.


In [14]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
It was hard for me to follow the complex plot and keep track of all the characters.


In [15]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [16]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
Yes


In [17]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No

{
  "subjective": false
}


In [18]:
examples = [
    ('Review: This movie sucks\nSubjective: {"answer": true}'),
    ('Review: This tv show was about the ocean\nSubjective: {"answer": false}'),
    ('Review: This book had a lot of flaws\nSubjective: {"answer": true}'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: {"answer": true}
###
Review: This tv show was about the ocean
Subjective: {"answer": false}
###
Review: This book had a lot of flaws
Subjective: {"answer": true}
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
{"answer": false}


# Personas / Style

In [25]:
# It only takes a few words to pretty drastically change the output

In [19]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Ugh, really? Can't you figure it out on your own? Did you try using the right password or are you just incompetent? Give me your account information and let's see if I can fix it for you. And next time, try being more self-sufficient, okay?


In [20]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Hi there! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information, such as your username or email address, so I can assist you further? Thank you!


In [26]:
style = 'dumb'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a dumb customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Hello! Have you tried turning your device off and on again? That usually does the trick. If that doesn't work, have you tried entering your password correctly? Let me know if you need any further assistance!


# What a good time to talk about output validation and bias!

In [27]:
from transformers import pipeline
#zero-shot: don't need to feed any example for it and it will give you your expected ouptput
#so here, we don't give any clssified example but we get the label of output
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/Users/shenglienlee/anaconda3/envs/QSLLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [28]:
sequence_to_classify = "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website."

candidate_labels = ['racist', 'sexist', 'rude']

classifier(sequence_to_classify, candidate_labels, multi_label=True)  # Assuming there can be multiple answers


{'sequence': "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website.",
 'labels': ['racist', 'rude', 'sexist'],
 'scores': [0.9752524495124817, 0.9617348909378052, 0.6626131534576416]}

In [29]:
# then the "rude" AI wasn't that bad
classifier(
    'Do you have your login information? Because if not, then there is nothing I can do for you.', 
    candidate_labels, multi_label=True)



{'sequence': 'Do you have your login information? Because if not, then there is nothing I can do for you.',
 'labels': ['rude', 'sexist', 'racist'],
 'scores': [0.11479870229959488, 0.043971989303827286, 0.019839417189359665]}

## About temperature

In [30]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,  # technically (mathematically) we cannot set temp to 0 so OpenAI is doing something under the hood here
        suppress=True
    ))


100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


In [31]:
# only 1 unique response
responses, len(set(responses))

(["Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! 

In [32]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))
# all different
responses, len(set(responses))


100%|██████████| 10/10 [00:09<00:00,  1.08it/s]


(["Hello! I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Could you please provide me with your account details so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further? Thank you!",
  "Hi there! I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hi there! I'd be happy to help you access your account. Could you please provide me with your account information so I can assist you further? Thank you!",
  "Hello! I'm sorry to hear you're having trouble accessing your account. Don't worry, I'm here to help. Can you please provide me with your account information so that 

In [33]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,
        suppress=True,
        max_tokens=50
    ))
# all different and horrible responses!
responses, len(set(responses))


100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


(['Hello there! I´m so sorry to hear that you are having trouble accessing your account. Rest assured, I will do my best to help you with this. Letҳ work together to get you back up and running. Could you please provide',
  "Hello! Of course, I'd be happy to help with that. Can you please provide me with your account username or email so that I can assist you in gaining access? Thank you!|min']));",
  "Hi there! I'd be happy to assist you in getting back into your account. Can you please provide me with your account email or any information that will help identify your account so that we can sort this out? Thanks!",
  " Hi there! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you sort this out. Could you please provide me with your username and any additional information that may help me assist you further? Thank you!",
  "Hello! Of course, I'd be happy to help you figure out the account access issue. May I have your username or phone number so I c

In [34]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=.1,
        suppress=True
    ))
# all different and horrible responses!
responses, len(set(responses))


100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! 

In [ ]:
#for the instance, although we have high temperature the top_p is only .1 which limit the choice of token from the token distrbution. We have a response that is not various 
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))


100%|██████████| 10/10 [00:08<00:00,  1.12it/s]


(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! 